# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os
from citipy import citipy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("city_data.csv")
data.head()

,Latitude,Longitude,City,Country,Max Temp,Humidity,Cloudiness,Date,Wind Speed
0,-54.80,-68.30,Ushuaia,AR,278.15,75,40,1601940667,5.10
1,-53.15,-70.92,Punta Arenas,CL,277.15,72,40,1601940667,7.20
2,59.70,30.79,Nikolskoye,RU,287.59,87,20,1601940667,3.00
3,22.08,-159.32,Kapaa,US,304.15,62,40,1601940667,5.10
4,-42.78,147.06,New Norfolk,AU,283.71,72,2,1601940668,3.58


In [3]:
# Correct temperature
farenheit = [(9/5) * (t-273) + 32 for t in data["Max Temp"]]
data["Max Temp"] = farenheit
data.head()

,Latitude,Longitude,City,Country,Max Temp,Humidity,Cloudiness,Date,Wind Speed
0,-54.80,-68.30,Ushuaia,AR,41.270,75,40,1601940667,5.10
1,-53.15,-70.92,Punta Arenas,CL,39.470,72,40,1601940667,7.20
2,59.70,30.79,Nikolskoye,RU,58.262,87,20,1601940667,3.00
3,22.08,-159.32,Kapaa,US,88.070,62,40,1601940667,5.10
4,-42.78,147.06,New Norfolk,AU,51.278,72,2,1601940668,3.58


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure!
gmaps.configure(api_key = g_key)

In [5]:
locations = data[["Latitude", "Longitude"]].astype(float)
humidity = data["Humidity"].astype(float)
maxhumidity = humidity.max()

In [6]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                   dissipating = False, max_intensity = 5,
                                   point_radius = 3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal = data.loc[((data["Max Temp"] <= 80) & (data["Max Temp"] >= 70)) & (data["Wind Speed"] < 10) & (data["Cloudiness"] == 0)]
ideal


,Latitude,Longitude,City,Country,Max Temp,Humidity,Cloudiness,Date,Wind Speed
115,25.35,73.31,Rāni,IN,76.514,55,0,1601940719,2.08
185,33.64,42.83,Hīt,IQ,74.300,21,0,1601940738,4.41
253,1.75,40.06,Wajir,KE,76.856,75,0,1601940716,5.60
304,6.12,81.12,Hambantota,LK,78.134,81,0,1601940769,6.29
401,12.39,-5.46,Koutiala,ML,77.198,81,0,1601940794,3.05
416,31.61,34.76,Kiryat Gat,IL,75.272,100,0,1601940797,1.83
451,-23.49,-58.80,Pozo Colorado,PY,75.794,44,0,1601940807,1.92
463,31.33,37.34,Qurayyat,SA,71.870,40,0,1601940810,2.10
493,27.80,79.80,Kānt,IN,73.130,46,0,1601940817,0.89
520,29.97,40.21,Sakakah,SA,70.070,22,0,1601940824,2.08


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_name = []
hotel_loc = ideal[["Latitude", "Longitude", "City", "Country"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
     "keyword": "hotel",
     "type": "hotel",
     "radius": 5000,
     "key": g_key,
 }

count = 1

for index, row in hotel_loc.iterrows():
    
    lat = row["Latitude"]
    lon = row["Longitude"]
    params["location"] = f"{lat},{lon}"
    print(f"Retrieving the data for hotel {count}.")
    count = count + 1
    hotel_response = requests.get(base_url, params = params)
    hotel_json = hotel_response.json()
    
    try:
        hotel_name.append(hotel_json["results"][0]["name"])
        
    except (KeyError, IndexError):
        hotel_name.append(np.nan)
        print("Hotel data not found. Thank you, next!")

    

Retrieving the data for hotel 1.
Hotel data not found. Thank you, next!
Retrieving the data for hotel 2.
Hotel data not found. Thank you, next!
Retrieving the data for hotel 3.
Retrieving the data for hotel 4.
Retrieving the data for hotel 5.
Retrieving the data for hotel 6.
Retrieving the data for hotel 7.
Retrieving the data for hotel 8.
Retrieving the data for hotel 9.
Hotel data not found. Thank you, next!
Retrieving the data for hotel 10.
Retrieving the data for hotel 11.
Hotel data not found. Thank you, next!


In [9]:
hotels = pd.read_csv("hotellist.csv")
hotels.dropna(subset = ["Hotel"], inplace = True)
hotels.reset_index(drop = True, inplace = True)
hotels

,Latitude,Longitude,City,Country,Hotel
0,6.12,81.12,Hambantota,LK,Jade Green Hotel Hambantota
1,-23.49,-58.80,Pozo Colorado,PY,Hospedaje la Amistad
2,29.97,40.21,Sakakah,SA,Raoum Inn Hotel


In [10]:
hotels.loc[:, ("Hotel")]
print(hotels.loc[:, ("Hotel")])

0    Jade Green Hotel Hambantota
1           Hospedaje la Amistad
2                Raoum Inn Hotel
Name: Hotel, dtype: object


In [11]:
# Export that stuff.
hotels.to_csv("hotellist.csv", index = False)

In [12]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotels = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotel_loc[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))